In [1]:
#Project DSP CSCE-3611/363_Spring 18
#Amr Gouhar       900153482
#Mohamed Ahmed    900153487
#Ahmed Zahed      900130311

from scipy.io import wavfile
import numpy as np
import cmath
#read the data and sample rate of the message
Rm, Dm = wavfile.read('message.wav');
#read the data and sample rate of the carrier
Rc, Dc = wavfile.read('carrier.wav');

#message length
Msz = len(Dm);
#carrier length
Csz = len(Dc);

#if the carrier's size is not enough to carry the length prompt the user 
if(Msz * 16 > Csz - 64):
    print("incryption not successful, carrier size not sufficient");
else:
    #declaring the bitstream
    Encryp = np.zeros((Msz * 16) + 64);
    #encrypt the size of the message
    for i in range(32):
        Encryp[i] = (Msz) & (1 << i);
    #encrypt the sample rate of the message
    for i in range(32,64):
        Encryp[i] = (Rm) & (1 << (i%32)); 
    #encrypt the message
    for i in range(64, Msz):
        for j in range(16):
            Encryp[i*16+j] = Dm[i - 64] & (1 << j);
    #add the encrypted size, sample rate, and message into the LSB of each data index
    for i in range((Msz*16)+64):
        if(Encryp[i]):
            Dc[i] |= 1;
        else:
            Dc[i] &= 0xfffe;


In [2]:
#prepare the data for writing into the audio file
out = np.asarray(Dc, dtype=np.int16);
#write the out file
wavfile.write('Zombie.wav', Rc, out);

In [3]:
from scipy.io import wavfile
import numpy as np
import cmath
#read the encrypted carrier audio file
Rs, Ds = wavfile.read('Zombie.wav');

#declare the size of the hidden message
Msz = 0;
#decrypt the message length
for i in range(32):
    Msz  |= ((Ds[i] & 1) << i);
#declare the message data and the sample rate
Do = [0]*(Msz);
Ro = 0;
#decrypt the sample rate
for i in range(32,64):
    Ro  |= ((Ds[i] & 1) << (i%32));
#decrypt the message
for i in range(64, Msz + 64):
    Do[(i-64)/16] = Do[(i-64)/16] | ((Ds[i] & 1) << ((i-64)%16));


In [4]:
#prepare the data for writing into the audio file
out = np.asarray(Dm, dtype=np.int16);
#write the out file
wavfile.write('hiddenMessage.wav', Ro, out);